In [1]:
import pandas
import numpy

In [2]:
d=pandas.read_csv("exercise-txn-db.tsv",sep="\t")

In [4]:
import timeit
#Takes about 30 seconds to "clean" formatting
exerciseslist=list()
start=timeit.default_timer()
for i in range(0,d.shape[0]):
    exerciseslist.append(d['exercises'].iloc[i].split(','))
#print(timeit.default_timer()-start)

In [6]:
from pyspark import SparkContext
from pyspark.mllib.fpm import *
sc = SparkContext("local", "App Name")
rdd=sc.parallelize(exerciseslist, 2)

In [7]:
modelhalfperc=FPGrowth.train(rdd, 0.005, 2)
modeltwoperc=FPGrowth.train(rdd, 0.02, 2)
modelfiveperc=FPGrowth.train(rdd, 0.05, 2)
modeltenperc=FPGrowth.train(rdd, 0.1, 2)
modelhalf=FPGrowth.train(rdd, 0.5, 2)

In [21]:
len(modelhalfperc.freqItemsets().collect())

429

In [22]:
len(modeltwoperc.freqItemsets().collect())

72

In [20]:
len(modelfiveperc.freqItemsets().collect())

17

In [23]:
len(modeltenperc.freqItemsets().collect())

2

In [24]:
len(modelhalf.freqItemsets().collect())

0

In [8]:
freq2=pandas.DataFrame(modeltwoperc.freqItemsets().collect())

In [53]:
print(freq2.sort_values('freq',ascending=False)[0:20].reset_index(drop=True))

                                      items    freq
0                                 [Push-Up]  125438
1                     [Barbell Bench Press]  106541
2                        [Barbell Deadlift]   94368
3                                   [Plank]   93929
4                           [Barbell Squat]   93081
5                                  [Crunch]   89103
6                                 [Pull-Up]   87780
7                     [Dumbbell Bicep Curl]   86247
8                              [Stretching]   77906
9                       [Body Weight Squat]   70446
10                                [Walking]   70219
11  [Standing Barbell Shoulder Press (OHP)]   68239
12                           [Lat Pulldown]   57531
13                                [Chin-Up]   55917
14                              [Leg Press]   52056
15                                [Running]   51588
16                   [One-Arm Dumbbell Row]   51291
17                 [Dips - Triceps Version]   48938
18          

In [49]:
with pandas.option_context('display.max_rows', None):
    print(freq2.sort_values('freq',ascending=False))

                                                items    freq
35                                          [Push-Up]  125438
58                              [Barbell Bench Press]  106541
10                                 [Barbell Deadlift]   94368
66                                            [Plank]   93929
16                                    [Barbell Squat]   93081
46                                           [Crunch]   89103
34                                          [Pull-Up]   87780
53                              [Dumbbell Bicep Curl]   86247
3                                        [Stretching]   77906
62                                [Body Weight Squat]   70446
23                                          [Walking]   70219
42            [Standing Barbell Shoulder Press (OHP)]   68239
30                                     [Lat Pulldown]   57531
51                                          [Chin-Up]   55917
12                                        [Leg Press]   52056
69      

In [1]:
freqall=pandas.DataFrame(modelhalfperc.freqItemsets().collect())

NameError: name 'pandas' is not defined

In [37]:
#This could have been made into a function, 
#since we do it again for 3 items
#but was not due to time constraints

ind=list()
for i in range(0, freqall.shape[0]):
    if(len(freqall['items'][i])==2):
        ind.append(freqall.index[i])

In [38]:
twoitems=freqall.loc[ind,:]
twoitems['freq']=twoitems['freq'].astype('int')
twoitems=twoitems.sort_values('freq', ascending=False)
with pandas.option_context('display.max_rows', None):
    print(twoitems)

                                                 items   freq
397                                   [Plank, Push-Up]  27128
382                       [Body Weight Squat, Push-Up]  26080
274                                  [Crunch, Push-Up]  25852
272                                    [Crunch, Plank]  24340
89                   [Barbell Squat, Barbell Deadlift]  24081
256  [Standing Barbell Shoulder Press (OHP), Barbel...  23251
91                [Barbell Squat, Barbell Bench Press]  23083
58             [Barbell Deadlift, Barbell Bench Press]  21512
47                                 [Side Plank, Plank]  20711
222                        [Leg Extensions, Leg Press]  18524
379                         [Body Weight Squat, Plank]  17845
233                  [Seated Leg Curl, Leg Extensions]  16130
202                                 [Pull-Up, Push-Up]  15828
374                        [Body Weight Squat, Crunch]  15483
257  [Standing Barbell Shoulder Press (OHP), Barbel...  15307
100     

In [52]:
with pandas.option_context('display.max_colwidth', None):
    print(twoitems[twoitems['freq']==7154])

ValueError: Value must have type '<class 'int'>'

In [27]:
ind=list()
for i in range(0, freqall.shape[0]):
    if(len(freqall['items'][i])==3):
        ind.append(freqall.index[i])
threeitems=freqall.iloc[ind,:]
threeitems=threeitems.sort_values('freq', ascending=False)

In [28]:
threeitems['freq']=threeitems['freq'].astype('int')
print(threeitems)

                                                 items  freq
273                           [Crunch, Plank, Push-Up]  9367
380                [Body Weight Squat, Plank, Push-Up]  9012
234       [Seated Leg Curl, Leg Extensions, Leg Press]  8102
407    [Body Weight Lunge, Body Weight Squat, Push-Up]  7889
376               [Body Weight Squat, Crunch, Push-Up]  7427
90   [Barbell Squat, Barbell Deadlift, Barbell Benc...  7122
375                 [Body Weight Squat, Crunch, Plank]  6869
258  [Standing Barbell Shoulder Press (OHP), Barbel...  6511
183        [Jumping Jacks, Body Weight Squat, Push-Up]  6402
185                    [Jumping Jacks, Plank, Push-Up]  6305
421                           [Bicycle, Crunch, Plank]  6236
48                        [Side Plank, Plank, Push-Up]  6129
406      [Body Weight Lunge, Body Weight Squat, Plank]  5992
173                  [ Group C ,  Group B ,  Group A ]  5821
45                         [Side Plank, Crunch, Plank]  5737
182          [Jumping Ja

In [25]:
ind=list()
for i in range(0, freqall.shape[0]):
    if(len(freqall['items'][i])>3):
        ind.append(freqall.index[i])
morethan3=freqall.iloc[ind,:]
len(morethan3)

0

In [54]:
print(twoitems.head(10))

                                                 items   freq
397                                   [Plank, Push-Up]  27128
382                       [Body Weight Squat, Push-Up]  26080
274                                  [Crunch, Push-Up]  25852
272                                    [Crunch, Plank]  24340
89                   [Barbell Squat, Barbell Deadlift]  24081
256  [Standing Barbell Shoulder Press (OHP), Barbel...  23251
91                [Barbell Squat, Barbell Bench Press]  23083
58             [Barbell Deadlift, Barbell Bench Press]  21512
47                                 [Side Plank, Plank]  20711
222                        [Leg Extensions, Leg Press]  18524


In [55]:
print(twoitems.tail(10))

                                                 items  freq
215                 [Adductor Machine, Leg Extensions]  5126
362  [Calf Press On The Leg Press Machine, Leg Exte...  5125
332                       [Running (treadmill), Plank]  5110
157               [Dumbbell Side Lateral Raise, Plank]  5097
195                       [Barbell Curl, Lat Pulldown]  5070
388          [Bent Over Barbell Row, Seated Cable Row]  5068
165                             [Lat Pulldown, Crunch]  5055
99                                   [Sit-Up, Pull-Up]  5026
166                      [Lat Pulldown, Barbell Squat]  5024
289                          [Dumbbell Squat, Push-Up]  5022


In [56]:
len(twoitems)

216